In [1]:
import torch
import torch.nn as nn
import pdb

import os
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_teng_data(sensor):
    i=0
    while i<sensor.shape[0]:
        teng_data_block = sensor[i:i+10]
        if i==0:
            teng_data = teng_data_block
        else:
            teng_data = torch.cat((teng_data,teng_data_block),dim=0)
        i+=28 # jump (28-10=18)
    return teng_data

def read_imu_data(sensor):
    i=10
    while i<sensor.shape[0]:
        imu_data_block = sensor[i:i+18]
        if i==10:
            imu_data = imu_data_block
        else:
            imu_data = torch.cat((imu_data,imu_data_block),dim=0)
        i+=28 # jump (28-18=10)
    return imu_data

In [ ]:
import pandas as pd

folder_path = '/mnt/fyp/raw_data'

for i,j in enumerate(sorted(os.listdir(folder_path))):
    sub_folder_path = os.path.join(folder_path,j)
    sensor_path = sorted(os.listdir(sub_folder_path))[0]
    sensor_path = os.path.join(sub_folder_path, sensor_path)
    try: # 如果sensor文件格式是csv_text
        sensor = pd.read_csv(sensor_path, header=None)
    except: # 如果sensor文件格式是Microsoft Excel
        sensor = pd.read_excel(sensor_path, engine='xlrd', header=None)
    
    sensor = sensor.applymap(lambda x: float(str(x).split('.')[0] + '.' + str(x).split('.')[1]) if '.' in str(x) else x)
    sensor = sensor.astype("float32")
    sensor = torch.tensor(sensor.values)

    teng_data_per_category = read_teng_data(sensor) 
    imu_data_per_category = read_imu_data(sensor)
    
    if i==0:
        teng_data = teng_data_per_category
        imu_data = imu_data_per_category
    else:
        teng_data = torch.cat((teng_data,teng_data_per_category),dim=1)
        imu_data = torch.cat((imu_data,imu_data_per_category),dim=1)

teng_data = teng_data.t()
imu_data = imu_data.t()

In [ ]:
torch.save(teng_data,"/mnt/fyp/data/teng_data.pt")
torch.save(imu_data,"/mnt/fyp/data/imu_data.pt")